In [ ]:
import os 
import fastf1
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# enable cache to locally store data, so that when load data don't need to take them every time from the api 
cache_dir = os.path.expanduser('~/fastf1_cache')
os.makedirs(cache_dir, exist_ok=True)
fastf1.Cache.enable_cache(cache_dir)

# I don't know if this is needed considering what we are doing now 

In [ ]:
# consider doing some work on data before actually appending it to the csv -> take from 3 practice session the fastest lap, 
# try to estimate the average time during a stint in practice 
# also add to dataset the Results data -> using SessionResults

In [18]:
def save_driver_session_data(new_df, event_name, year):
    output_folder = "./data"
    output_file = os.path.join(output_folder, f"f1_data_{event_name.replace(' ', '_')}_{year}.csv")

    os.makedirs(output_folder, exist_ok=True)

    if os.path.isfile(output_file):
        # Load existing data
        existing_df = pd.read_csv(output_file)

        # Merge on Driver, keeping all data
        combined_df = pd.merge(
            existing_df,
            new_df,
            on='Driver',
            how='outer',
            suffixes=('', '_dup')
        )

        # Remove duplicate columns caused by merge (e.g., if same column already existed)
        combined_df = combined_df.loc[:, ~combined_df.columns.str.endswith('_dup')]

    else:
        combined_df = new_df

    # Save the merged result
    combined_df.to_csv(output_file, index=False)
    print(f"Data saved to: {output_file}")

In [33]:
def get_practice_data(year, event_name, session_name="FP1"):

    # Load session data
    session = fastf1.get_session(year, event_name, session_name)
    session.load(laps=True)
    laps = session.laps.copy()

    # Get fastest lap for each driver using IsPersonalBest
    fastest_laps = laps[laps["IsPersonalBest"]==True].groupby("Driver").agg({
        'LapTime': 'min',
        'Compound': 'first',
        'SpeedST': 'max',
        'SpeedFL': 'max',
        'SpeedI1': 'max',
        'SpeedI2': 'max'
    }).reset_index().rename(columns=lambda x: x if x == 'Driver' else f"{x}_{session_name}")

    # Get weather data for each lap
    weather_data = laps.get_weather_data().reset_index(drop=True)
    laps_weather = pd.concat([laps.reset_index(drop=True), weather_data.drop(columns=['Time'])], axis=1)

    # Identify longest stint per driver and compute average lap time, delta, tyre life, and weather data
    stint_data = []
    for driver in laps['Driver'].unique():
        driver_laps = laps_weather[laps_weather['Driver'] == driver]
        longest_stint = driver_laps.groupby(['Stint', 'Compound']).size().reset_index(name='Laps').sort_values('Laps', ascending=False).iloc[0]

        stint_laps = driver_laps[(driver_laps['Stint'] == longest_stint['Stint']) & (driver_laps['Compound'] == longest_stint['Compound'])]
        stint_laps_sorted = stint_laps.sort_values('LapNumber')
        avg_stint_time = stint_laps_sorted['LapTime'].mean()
        delta_time = stint_laps_sorted['LapTime'].iloc[-1] - stint_laps_sorted['LapTime'].iloc[0]

        invalid_laps = stint_laps_sorted['Deleted'].sum()

        stint_data.append({
            'Driver': driver,
            f'LongestStintCompound_{session_name}': longest_stint['Compound'],
            f'LongestStintLaps_{session_name}': longest_stint['Laps'],
            f'AvgLapTime_LongestStint_{session_name}': avg_stint_time,
            f'Delta_FirstLastLap_{session_name}': delta_time,
            f'InvalidLaps_LongestStint_{session_name}': invalid_laps,
            f'AvgTyreLife_LongestStint_{session_name}': stint_laps_sorted['TyreLife'].mean(),
            f'StartTyreLife_LongestStint_{session_name}': stint_laps_sorted['TyreLife'].iloc[0],
            f'AvgAirTemp_LongestStint_{session_name}': stint_laps_sorted['AirTemp'].mean(),
            f'AvgTrackTemp_LongestStint_{session_name}': stint_laps_sorted['TrackTemp'].mean(),
            f'AvgWindSpeed_LongestStint_{session_name}': stint_laps_sorted['WindSpeed'].mean(),
            f'AvgWindDirection_LongestStint_{session_name}': stint_laps_sorted['WindDirection'].mean(),
            f'AvgHumidity_LongestStint_{session_name}': stint_laps_sorted['Humidity'].mean(),
            f'AvgPressure_LongestStint_{session_name}': stint_laps_sorted['Pressure'].mean()
        })

    stint_df = pd.DataFrame(stint_data)

   # Keep only the fastest personal best lap weather per driver
    fastest_weather = laps_weather[laps_weather["IsPersonalBest"] == True].sort_values('LapTime').drop_duplicates(subset=['Driver'], keep='first')[
        ['Driver', 'AirTemp', 'TrackTemp', 'WindSpeed', 'WindDirection', 'Humidity', 'Pressure']
    ].rename(columns=lambda x: x if x == 'Driver' else f"{x}_FastestLap_{session_name}")

    # Merge fastest laps, stint data, and weather data
    driver_session_summary = fastest_laps.merge(stint_df, on='Driver', how='left')
    driver_session_summary = driver_session_summary.merge(fastest_weather, on='Driver', how='left')

    save_driver_session_data(driver_session_summary, event_name, year)

In [20]:
def get_quali_data(year, event_name):
    session = fastf1.get_session(year, event_name, 'Q')
    session.load(laps=True)
    results = session.results

    qualifying_data = []

    weather_data = session.laps.get_weather_data().reset_index(drop=True)
    laps_weather = pd.concat([session.laps.reset_index(drop=True), weather_data.drop(columns=['Time'])], axis=1)

    for _, row in results.iterrows():
        driver = row['Abbreviation']

        driver_info = {
            'Driver': driver,
            'Q1_FastestLap': row['Q1'],
            'Q2_FastestLap': row['Q2'],
            'Q3_FastestLap': row['Q3'],
            'QualifyingPosition': row['Position']
        }

        for q, q_time in zip(['Q1', 'Q2', 'Q3'], [row['Q1'], row['Q2'], row['Q3']]):
            if pd.isnull(q_time):
                driver_info[f'{q}_TopSpeedST'] = None
                driver_info[f'{q}_AirTemp'] = None
                driver_info[f'{q}_TrackTemp'] = None
                driver_info[f'{q}_WindSpeed'] = None
                driver_info[f'{q}_WindDirection'] = None
                driver_info[f'{q}_Humidity'] = None
                driver_info[f'{q}_Pressure'] = None
            else:
                lap_data = laps_weather[(laps_weather['Driver'] == driver) & (laps_weather['LapTime'] == q_time)]
                if not lap_data.empty:
                    lap = lap_data.iloc[0]
                    driver_info[f'{q}_TopSpeedST'] = lap['SpeedST']
                    driver_info[f'{q}_AirTemp'] = lap['AirTemp']
                    driver_info[f'{q}_TrackTemp'] = lap['TrackTemp']
                    driver_info[f'{q}_WindSpeed'] = lap['WindSpeed']
                    driver_info[f'{q}_WindDirection'] = lap['WindDirection']
                    driver_info[f'{q}_Humidity'] = lap['Humidity']
                    driver_info[f'{q}_Pressure'] = lap['Pressure']
                else:
                    driver_info[f'{q}_TopSpeedST'] = None
                    driver_info[f'{q}_AirTemp'] = None
                    driver_info[f'{q}_TrackTemp'] = None
                    driver_info[f'{q}_WindSpeed'] = None
                    driver_info[f'{q}_WindDirection'] = None
                    driver_info[f'{q}_Humidity'] = None
                    driver_info[f'{q}_Pressure'] = None

        qualifying_data.append(driver_info)

    qualifying_df = pd.DataFrame(qualifying_data)

    save_driver_session_data(qualifying_df, event_name, year)

    

In [ ]:
def get_race_result(year, event_name):
    # Load race session
    session = fastf1.get_session(year, event_name, 'Race')
    session.load()

    # Extract relevant data
    results = session.results[["Abbreviation", "Time", "Position"]].copy()
    results.rename(columns={
        "Time": "TotalRaceTime",
        "Position": "RacePosition",
        "Abbreviation": "Driver"
    }, inplace=True)

    # Merge and save
    save_driver_session_data(results, event_name, year)

In [28]:
def load_event_data(year, event_name):
    get_practice_data(year, event_name, "FP1")
    get_practice_data(year, event_name, "FP2")
    get_practice_data(year, event_name, "FP3")
    get_quali_data(year, event_name)
    get_race_result(year, event_name) 

In [30]:
load_event_data(2025, "Australian Grand Prix")
load_event_data(2025, "Japanese Grand Prix")

core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '7', '10', '12', '14', '16', '18', '22', '23', '27', '30', '31', '44', '55', '63', '81', '87']
core           INFO 	Loading data for Australian Grand Prix - Practice 2 [v3.5.3]
req            INFO 	Usin

Data saved to: ./data/f1_data_Australian_Grand_Prix_2025.csv


core        WARNING 	No lap data for driver 87
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 87)
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '7', '10', '12', '14', '16', '18', '22', '23', '27', '30', '31', '44', '55', '63', '81', '87']
core           INFO 	Loading data for Australian Grand Prix - Practice 3 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core 

Data saved to: ./data/f1_data_Australian_Grand_Prix_2025.csv


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '7', '10', '12', '14', '16', '18', '22', '23', '27', '30', '31', '44', '55', '63', '81', '87']
core           INFO 	Loading data for Australian Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


...

Data saved to: ./data/f1_data_Japanese_Grand_Prix_2025.csv


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
req            INFO 	No cached data found for car_data. Loading data...
_api           INFO 	Fetching car data...
_api

Data saved to: ./data/f1_data_Japanese_Grand_Prix_2025.csv


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written t

Data saved to: ./data/f1_data_Japanese_Grand_Prix_2025.csv


In [ ]:
# load 20024 race data for races which were not sprint weekends
#load_event_data(2024, "Bahrain Grand Prix")
#load_event_data(2024, "Saudi Arabian Grand Prix")
#load_event_data(2024, "Australian Grand Prix")
#load_event_data(2024, "Japanese Grand Prix")
#load_event_data(2024, "Emilia Romagna Grand Prix")
#load_event_data(2024, "Monaco Grand Prix")
#load_event_data(2024, "Canadian Grand Prix")
#load_event_data(2024, "Spain Grand Prix")
#load_event_data(2024, "British Grand Prix")
load_event_data(2024, "Hungarian Grand Prix")
load_event_data(2024, "Belgian Grand Prix")
load_event_data(2024, "Dutch Grand Prix")
load_event_data(2024, "Italian Grand Prix")
load_event_data(2024, "Azerbaijan Grand Prix")
load_event_data(2024, "Singapore Grand Prix")
load_event_data(2024, "Mexican Grand Prix")
load_event_data(2024, "Las Vegas Grand Prix")
load_event_data(2024, "Abu Dhabi Grand Prix")

core           INFO 	Loading data for Bahrain Grand Prix - Practice 1 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '2', '3', '4', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '31', '44', '55', '63', '77', '81']
core           INFO 	Loading data for Bahrain Grand Prix - Practice 2 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for tra

Data saved to: ./data/f1_data_Bahrain_Grand_Prix_2024.csv


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '2', '3', '4', '10', '11', '14', '16', '18', '20', '22', '23', '24', '27', '31', '44', '55', '63', '77', '81']
core           INFO 	Loading data for Bahrain Grand Prix - Practice 3 [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Data saved to: ./data/f1_data_Bahrain_Grand_Prix_2024.csv


...

Data saved to: ./data/f1_data_British_Grand_Prix_2024.csv


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

Data saved to: ./data/f1_data_Hungarian_Grand_Prix_2024.csv


core        WARNING 	Failed to load extended driver information!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNING 	Failed to load track status data!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
logger      WARNING 	Failed to load timing data!
req            INFO 	No cached data found for car_data. Loading data...
_api           INFO 	Fetching car data...
logger      WARNING 	Failed to load telemetry data!
req            INFO 	No cached data found for weather_data. Loading data...
_api           INFO 	Fetching weather data...
logger      WARNING 	Failed to load weather data!
req            INFO 	No cached data fo

DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`